# Random Forest

Si classifica nei metodi d'insieme e che fa uso del ***Bagging***. Non ha la componente di metodo d'insieme incorporata, come nel Boosting.

Il random forest utilizza l'albero decisionale binario come classificatore base. Il random forest è un metodo di classificazione che consiste nella combinazione di più alberi decisionali. Ogni albero decisionale è costruito su un sottoinsieme di dati estratti casualmente dal dataset originale. Inoltre, per ogni nodo dell'albero, viene selezionato un sottoinsieme casuale di variabili. Il risultato finale è ottenuto dalla media delle previsioni di tutti gli alberi.

L'albero a ogni nodo "pone un quesito" e tramite esso scelgo una strada fino ad arrivare a una foglia che corrisponde alla classificazione.

Si dice che è un modello completamente interpretabile, perché andando a ritroso, posso ricostruire le decisioni che ha fatto il classificatore.

---


# Binary Decision Trees

- dati rappresentati come feature vector: $v \in \mathbb{R}^N$
- <span style="color:gold;">funzione di split</span> del feature vector $f_n(v): \mathbb{R}^N \rightarrow \mathbb{R}$ che ritorna uno scalare equivalente alla domanda che pongo al nodo
- Una soglia $t_n \in \mathbb{R}$ che rappresenta il valore di soglia per la funzione di split: se sono sotto la soglia vado a sinistra e sopra la soglia a destra (o il contrario)
- Classificazioni $P_n(c)$ per ogni nodo $n$ e per ogni classe $c$ che rappresenta la probabilità di appartenenza alla classe $c$ per il nodo $n$.


Durante il training dobbiamo imparare la funzione di split per ogni nodo e la soglia. 

Il classificatore ha per ogni nodo una funzione di split e un valore di soglia.

Nelle foglie non c'è un valore ma un istogramma (che viene costruito durante il training): ho sull'asse delle x le classi del mio problema e sull'asse delle y ho il numero degli elementi di training che hanno seguito quel percorso e sono finite in quelle foglie con un preciso valore di classe.

![06_Random_Forest1](./images/06_Random_Forest1.png)

In fase di test, uso l'istogramma per definire le probabilità che dato un elemento finito in quella foglia tramite un processo decisionale faccia parte delle varie classi.

---


# Principio di funzionamento

Esempio con:
- I vettori delle feature sono delle coordinate: <span style="color:red;">***$v = [x,y]^T$***</span>
- La label è il colore del punto: <span style="color:red;">rosso</span>, <span style="color:blue;">blu</span>, <span style="color:green;">verde</span>, <span style="color:purple;">viola</span>
- le funzioni di split sono linee con i parametri $a$, $b$: <span style="color:red;">***$f_n(v) = a x + b y$***</span>
- <span style="color:red;">***$t_n$***</span> rappresenta il valore di soglia per la funzione di split

Il primo split prova varie linee e deve tenere la migliore, ovvero una linea che nel suo split mi manda a destra e a sinistra dell'albero le classi senza spezzarne nessuna a metà

![06_Random_Forest2](./images/06_Random_Forest2.png)

Il nodo successivo cercherà di dividere le classi con il suo classificatore lineare

![06_Random_Forest3](./images/06_Random_Forest3.png)

Fino a che non si sono separati tutti i punti

![06_Random_Forest4](./images/06_Random_Forest4.png)

Quindi ogni classificatore binario cerca di risolvere un problema, ovvero cercare di mettere tutti gli elementi della stessa classe da una parte dello split

In questo caso alcuni punti verdi sono insieme ai punti blu, quindi i punti blu nell'istogramma avranno anche una probabilità di essere verdi

---






# Randomized Learning

Vado nello split di sinistra se:

### $I_{left} = \{i \in I | f_n(v_i) < t\}$

Vado nello split di destra se:

### $I_{right} = \{i \in I | f_n(v_i) \geq t\}$

---


# Extreme Randomized Tree (prende una feature a caso e una soglia a caso)

Come imparare la funzione di split?

È possibile scegliere una feature a caso del mio feature vector, guarda qual è il range nel dataset di quelle feature. Battezza un  po' di soglie all'interno del range, valuta la funzione di split e poi sceglie la feature e la soglia che massimizzano questa quantità:

### Information gain = $\delta E = - \frac{|I_l|}{|I_n|}E(I_l) - \frac{|I_r|}{|I_n|}E(I_r)$

$\frac{|I_1|}{|I_n|}$ misura quanti elementi del mio training sono finiti applicando i parametri che sto valutando nello split di sinistra diviso il numero degli elementi che sto cercando di splittare.

$\frac{|I_2|}{|I_n|}$ misura quanti elementi del mio training sono finiti applicando i parametri che sto valutando nello split di destra diviso il numero degli elementi che sto cercando di splittare.


Quindi:
- so che ho una quantità di elementi che finiscono in un nodo
- battezzo una feature
- battezzo una quantità di soglie 
- applico la prima soglia e alcuni elementi finiscono a sinistra e altri a destra
- li conto e calcolo quei due coefficienti
- poi si deve calcolare l'entropia, rispettivamente $E(I_l)$ e $E(I_r)$ per l'entropia dello split di sinistra e destra

    dove l'entropia è $P(x)\log(x)$ con integrale o somma a seconda che siamo nel discreto o nel continuo

Nello specifico si sta calcolando l'entropia delle label, perché voglio cercare ***il valore di soglia che massimizza l'information gain***.

Alla fine in ogni split faccio l'istogramma delle label, cioè <span style="color:gold">conto quanti elementi di quello split fanno parte di una determinata classe</span>

In questo caso calcolo l'entropia come:

### $E = - \sum_i^{nlabel} h_i \log_2 (h_i)$

Ovvero la somma del valore della barretta per il logaritmo in base 2 di se stesso. Questo su tutte le barrette.

Massimizzare l'information gain $\delta E$ equivale a minimizzare l'entropia $E$.

L'entropia è minima se nell'istogramma avessi un'unica barretta e tutte le altre fossero a zero &rarr; in quel caso vale 0.

L'entropia è massima se tutte le barrette sono alte uguali, cioè tutte le classi sono equiprobabili.

Applicativamente, per allenare un nodo posso fare un algoritmo ricorsivo sui dati che sono finiti nello split di sinistra e sui dati che sono finiti nello split di destra.

---


# Dettagli di implementazione

Come iperparametri posso scegliere il numero di split da mettere, il numero di feature e di soglie da provare.

![06_Random_Forest5](./images/06_Random_Forest5.png)

Quando smettere di far crescere l'albero?
- posso fissare la profondità massima
- posso osservare il guadagno dell'entropia (se ho separato tutti gli elementi di una classe e continuo a separare avrò un peggioramento dell'information gain)
- pruning: fisso a mano la profondità del mio albero (decido quanti split fare) poi una volta trainato tolgo dal basso dei livelli e guardo se l'accuracy cambia. Se non cambia, quel livello è inutile e posso toglierlo 

---


# Randomized learning pseudo code

![06_Random_Forest6](./images/06_Random_Forest6.png)

---


Come faccio a trasformare questo albero decisionale in un classificatore che usa i metodi d'insieme?

Posso fare un bagging di alberi &rarr; ogni albero mi produce un istogramma, li sommo tutti, li normalizzo

Il random forest è proprio un bag di alberi decisionali binari

È random per due motivi:
- per la scelta degli elementi che finiscono in ogni bag
- nella scelta delle feature e delle soglie

Se scelgo le feature a caso e le soglie a caso, il mio albero è un random forest, ed è pure improbabile che due alberi abbiano le stesse funzioni di split &rarr; permette di non cadere negli errori e di avere tutti i benefici del bagging.


La decisione finale è la somma normalizzata dei vari istogrammi 

![06_Random_Forest7](./images/06_Random_Forest7.png)

---


# Learning a forest

Come faccio i bag?

- Posso scegliere i singoli elementi da mettere nei bag a mano o a caso
- Posso fare dei bag con l'overlap: un elemento può finire in più bag
- Posso fare dei bag a livello di feature: stessi elementi nelle varie feature ma con feature diverse
- Posso fare delle bag non uniformi